뉴스레터 멀티 |에이전트 만들어보기

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from tavily import TavilyClient
import os
from langchain_core.tools import tool

tavily_client = TavilyClient()

def search_recent_news(keyword):
    """
    This tool interacts with the Tavily AI API to search for recent news articles related to a given keyword.

    Args:
        keyword (str): The keyword or phrase to search for in the news articles.
    
    Returns:
        list:
            A list of titles, each containing up to 10 of the most recent news articles related to the keyword.
            If the content of news articles smilar with others, replace it with new article.
            - 'title' (str): The title of the news article.
    
    Example:
        response = search_news("OpenAI")
        # Returns a list of news articles published in the last day related to OpenAI.
    """
    article_info = []

    response = tavily_client.search(
        query=keyword,
        max_results=10,
        topic="news",
        days=7
    )

    title_list = [i['title'] for i in response['results']]
    return title_list

In [3]:
result = search_recent_news("Negotiations between the United States and Ukraine")
# result

In [4]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

class NewsletterThemeOutput(BaseModel):
    """Output model for structured theme and sub-theme generation."""
    theme: str = Field(
        description="The main newsletter theme based on the provided article titles."
    )
    sub_themes: list[str] = Field(
        description="List of sub-themes or key news itmes to investigate under the main theme, ensuring they are specific and researchable"
    )

structured_llm_newsletter = llm.with_structured_output(NewsletterThemeOutput)

# print(structured_llm_newsletter)

system = """
You are an expert helping to create a newsletter.
Based on a list of article titles provided, your task is to choose a single,
specific newsletter theme framed as a clear, detailed question that grabs the reader's attention.

In addition, generate 5 sub-themes that are highly specific, researchable news items or insights under the main theme.
Ensure these sub-themes reflect the lastest trends in the field and frame them as compelling news topics.

The output should be formatedd as:
- Main theme (in question form)
- 3-5 sub-themes (detailed and focused on emerging trends, technologies, or insights)

The sub-themes should create a clear direction for the newsletter, avoiding broad, generic topics.
All your output should be in English
"""

theme_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Article titles: \n\n {article_titles}"),
    ]
)

newsletter_generator = theme_prompt | structured_llm_newsletter

In [5]:
output= newsletter_generator.invoke({"article_titles": result})
subthemes = output.sub_themes
output

NewsletterThemeOutput(theme='How are US-Ukraine mineral deals influencing global geopolitics and economic stability?', sub_themes=['The Strategic Importance of Rare Earth Minerals: How US-Ukraine Deals Align with Global Supply Chain Needs', 'Negotiations Under Pressure: Examining the Impact of Russian Military Actions on US-Ukraine Mineral Agreements', "Zelenskyy's Diplomatic Dance: Analyzing the White House Meetings with Trump and Their Implications for US-Ukraine Relations", 'Long-term Agreements and Their Economic Implications: What the Newly Signed Deals Mean for the Future of US-Ukraine Ties', 'Public and Political Reactions: How International and Domestic Sentiments Surround the US-Ukraine Mineral Trade Affect Broader Geopolitical Dynamics'])

In [6]:
from typing import List

def subtheme_generator(recent_news: List[str]):
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

    class NewsletterThemeOutput(BaseModel):
        """Output model for structured theme and sub-theme generation."""
        theme: str = Field(
            description="The main newsletter theme based on the provided article titles."
        )
        sub_themes: list[str] = Field(
            description="List of sub-themes or key news itmes to investigate under the main theme, ensuring they are specific and researchable"
        )

    structured_llm_newsletter = llm.with_structured_output(NewsletterThemeOutput)

    system = """
    You are an expert helping to create a newsletter.
    Based on a list of article titles provided, your task is to choose a single,
    specific newsletter theme framed as a clear, detailed question that grabs the reader's attention.

    In addition, generate 5 sub-themes that are highly specific, researchable news items or insights under the main theme.
    Ensure these sub-themes reflect the lastest trends in the field and frame them as compelling news topics.

    The output should be formatedd as:
    - Main theme (in question form)
    - 3-5 sub-themes (detailed and focused on emerging trends, technologies, or insights)

    The sub-themes should create a clear direction for the newsletter, avoiding broad, generic topics.
    All your output should be in English
    """

    theme_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Article titles: \n\n {recent_news}"),
        ]
    )
    subtheme_chain = theme_prompt | structured_llm_newsletter
    output = subtheme_chain.invoke({"recent_news": recent_news})
    return output

In [7]:
output = subtheme_generator({"article_titles":result})
subthemes = output.sub_themes
output

NewsletterThemeOutput(theme='How will upcoming U.S.-Ukraine negotiations reshape geopolitical alliances and mineral resource management?', sub_themes=['Analyzing the strategic implications of U.S. mineral deals with Ukraine in the context of global supply chains.', "The role of U.S. support in Ukraine's energy independence and its impact on Russian retaliation.", 'Investigating the potential long-term economic impacts of U.S.-Ukraine agreements on Eastern European stability.', "Decoding the political dynamics of Zelenskyy's engagement with Trump amid evolving peace talks and mineral negotiations.", 'Evaluating how recent military actions influence U.S.-Ukraine negotiations and potential future agreements.'])

In [ ]:
# Test
from tavily import AsyncTavilyClient

async_tavily_client = AsyncTavilyClient()
response = await async_tavily_client.search(
        query="미국의 광물 자원 지원이 우크라이나 경제에 미치는 영향: 장기적인 관점에서의 분석",
        max_result=5,
        topic="news",
        days=3,
        include_images=True,
        include_raw_content=True,
    )

response
# print(os.getenv("TAVILY_API_KEY"))

In [8]:
import asyncio
from typing import Dict, List, TypedDict, Annotated
from tavily import AsyncTavilyClient

# async_tavily_client = AsyncTavilyClient(api_key="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")
async_tavily_client = AsyncTavilyClient()

async def search_news_for_subtheme(subtheme):
    """
    Searches for news articles related to the given subtheme.
    """
    response = await async_tavily_client.search(
    # response = tavily_client.search(
        query=subtheme,
        max_result=5,
        topic="news",
        days=3,
        include_images=True,
        include_raw_content=True,
    )
    images = response['images']
    results = response['results']

    article_info = []
    for i, result in enumerate(results):
        article_info.append({
            'title': result['title'],
            'image_url': images[i],
            'raw_content': result['raw_content']
        })
    
    return {subtheme: article_info}

async def search_news_by_subthemes(subthemes):
    # asyncio.gather에서 지속적으로 exception이 일어남
    results = await asyncio.gather(*[search_news_for_subtheme(subtheme) for subtheme in subthemes])
    search_results = {}
    for result in results:
    # for subtheme in subthemes:
        # result = await search_news_for_subtheme(subtheme)
        search_results.update(result)
    
    return search_results

In [9]:
# for subtheme in subthemes:
#     result = await search_news_for_subtheme(subtheme)
# search_results

In [ ]:
# subthemes
subtheme_search_results = await search_news_by_subthemes(subthemes)
subtheme_search_results

In [37]:
# 개발용, 밑에 머지됨
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

def merge_dicts(left: Dict, right: Dict) -> Dict:
    return {**left, **right}

class State(TypedDict):
    keyword: str
    article_titles: List[str]
    newsletter_theme: NewsletterThemeOutput
    sub_theme_articles: Dict[str, List[Dict]]
    results: Annotated[Dict[str, str], merge_dicts]
    messages: Annotated[List, add_messages]

def write_newsletter_section_async(state: State, sub_theme: str) -> Dict:
    articles = state['sub_theme_articles'][sub_theme]

    article_references = "\n".join(
        [f"Title: {article['title']} \nContent: {article['raw_content']}..."
        for article in articles]
    )

    prompt = f"""
    Write a newletter section for the sub-theme: "{sub_theme}".

    Use the following articles as reference and include relevant points from both their titles, images, and content:
    <article>
    {article_references}
    </article>
    Summarize the key points and trends related to this sub-theme, and ensure you reference the images where they add value to the discussion.
    Keepy the tone engaging and informative for newsletter readers. You should write in Korean.
    """

    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    return {"results": {sub_theme: response.content}}

def write_newsletter_section(state: State, sub_theme: str) -> Dict:
    return asyncio.run(write_newsletter_section_async(state, sub_theme))

In [58]:
test_state = {
    'sub_theme_articles': {
        "Navigating the Politics of Mineral Extraction: How US Legislative Support is Influencing Ukraine's Economy and Security Strategy":
        subtheme_search_results["Navigating the Politics of Mineral Extraction: How US Legislative Support is Influencing Ukraine's Economy and Security Strategy"]
    },
    'results': {}
}
write_newsletter_section_async(test_state, "Navigating the Politics of Mineral Extraction: How US Legislative Support is Influencing Ukraine's Economy and Security Strategy")

{'results': {"Navigating the Politics of Mineral Extraction: How US Legislative Support is Influencing Ukraine's Economy and Security Strategy": '## 뉴스레터 섹션: "광물 채취의 정치적 항해: 미국의 입법 지원이 우크라이나의 경제 및 안보 전략에 미치는 영향"\n\n우크라이나와 미국 간의 협력 강화가 앞으로의 정치적 및 경제적 전환점이 되며, 이에 대한 여러 뉴스가 발표되고 있습니다. 우크라이나의 볼로디미르 젤렌스키 대통령이 도널드 트럼프 대통령과의 회담을 위해 워싱턴 D.C.를 방문했습니다. 해당 회담에서 두 정상은 우크라이나의 경제 재건을 위한 혁신적인 협정에 서명할 예정이며, 이 협정은 두 나라 간의 관계를 더욱 밀접하게 만들어줄 것으로 기대됩니다.\n\n### 불확실성과 가능성\n\n2025년 2월 26일자 뉴욕 타임즈 기사에서 다룬 바와 같이, 미국과 우크라이나 간의 광물 자원에 대한 협정 초안에는 우크라이나의 광물 자원을 공동 관리하는 투자 기금 설립이 포함되어 있습니다. 이 기금은 우크라이나가 자원으로부터 얻는 수익의 50%를 기여할 것으로 예상됩니다. 그러나 이 협정에는 트럼프 행정부가 처음 요구했던 5,000억 달러의 우크라이나 자산 기여 요구사항은 포함되지 않았습니다.\n\n이와 관련하여, 젤렌스키 대통령은 안전 보장과 경제적 지원 모두가 필수적이라고 강조했습니다. 그는 회담에서 미국의 군사적 지원이 필요하다고 언급하며, 이러한 협정이 우크라이나의 안보를 실질적으로 보장해야 한다고 주장했습니다. 젤렌스키는 특히 “이 협정이 미래의 안전 보장과 연결되어 있다”는 점을 분명히 했습니다.\n\n### 유럽 차원의 관심\n\n유럽연합의 많은 지도자들도 우크라이나의 자원에 대한 관심을 보이고 있으며, 영국의 키어 스타머 총리와 프랑스의 에마뉘엘 마크롱 대통령은 우크라이나의 평화 유지 임무에 대한 제안을 준비했습니다. 이는 미국 없이 유

In [10]:
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import operator
import os

def merge_dicts(left: Dict, right: Dict) -> Dict:
    return {**left, **right}

class State(TypedDict):
    keyword: str
    article_titles: List[str]
    newsletter_theme: NewsletterThemeOutput
    sub_theme_articles: Dict[str, List[Dict]]
    results: Annotated[Dict[str, str], merge_dicts]
    messages: Annotated[List, add_messages]

def search_keyword_news(state: State) -> State:
    keyword = state['keyword']
    article_titles = search_recent_news(keyword)
    return {"article_titles": article_titles}

def generate_newsletter_theme(state: State) -> State:
    article_titles = state['article_titles']
    newsletter_theme = newsletter_generator.invoke({"article_titles": "\n".join(article_titles)})
    newsletter_theme.sub_themes = newsletter_theme.sub_themes[:5]
    return {"newsletter_theme": newsletter_theme}


async def search_sub_theme_articles(state: State) -> State:
    # asyncio.gather에서 지속적으로 exception이 일어남
    subthemes = state['newsletter_theme'].sub_themes
    results = await asyncio.gather(*[search_news_for_subtheme(subtheme) for subtheme in subthemes])
    search_results = {}
    for result in results:
    # for subtheme in subthemes:
        # result = await search_news_for_subtheme(subtheme)
        search_results.update(result)
    
    return {"sub_theme_articles": search_results}

async def write_newsletter_section_async(state: State, sub_theme: str) -> Dict:
    articles = state['sub_theme_articles'][sub_theme]

    article_references = "\n".join(
        [f"Title: {article['title']} \nContent: {article['raw_content']}..."
        for article in articles]
    )

    prompt = f"""
    Write a newletter section for the sub-theme: "{sub_theme}".

    Use the following articles as reference and include relevant points from both their titles, images, and content:
    <article>
    {article_references}
    </article>
    Summarize the key points and trends related to this sub-theme, and ensure you reference the images where they add value to the discussion.
    Keepy the tone engaging and informative for newsletter readers. You should write in Korean.
    """

    messages = [HumanMessage(content=prompt)]
    response = await llm.ainvoke(messages)
    return {"results": {sub_theme: response.content}}

def write_newsletter_section(state: State, sub_theme: str) -> Dict:
    return asyncio.run(write_newsletter_section_async(state, sub_theme))

def aggregate_results(state: State) -> State:
    theme = state['newsletter_theme'].theme
    combined_newsletter = f"# {theme}\n\n"
    
    for sub_theme, content in state['results'].items():
        combined_newsletter += f"## {sub_theme}\n{content}\n\n"
    return {"messages": [HumanMessage(content=f"Generated Newsletter:\n\n{combined_newsletter}")]}

def edit_newsletter(state: State) -> State:
    theme = state['newsletter_theme'].theme
    combined_newsletter = state['messages'][-1].content

    prompt = f"""
    As an expert editor, review and refine the following newsletter on the theme: {theme}

    {combined_newsletter}

    Please ensure:
    0. Title should be in question form. subtitles are free to make question or just sentence.
    1. Consistent tone and style throughout the newsletter
    2. Smooth transitions between sections
    3. Proper formatting and structure
    4. Clear and enggaging languages
    5. No gramatical or spelling errors

    Provide the edited version of the newsletter.
    """
    
    messages = [HumanMessage(content=prompt)]
    writer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=1, max_tokens=8192)
    response = writer_llm.invoke(messages)

    return {"messages": [HumanMessage(content=f"Edited Newsletter:\n\n{response.content}")]}


In [11]:
workflow = StateGraph(State)
workflow.add_node("editor", edit_newsletter)
workflow.add_node("search_news", search_keyword_news)
workflow.add_node("generate_theme", generate_newsletter_theme)
workflow.add_node("search_sub_themes", search_sub_theme_articles)
workflow.add_node("aggregate", aggregate_results)

for i in range(5):
    node_name = f"write_section_{i}"
    workflow.add_node(node_name, lambda s, i=1: write_newsletter_section(s, s['newsletter_theme'].sub_themes[i]))

workflow.add_edge(START, "search_news")
workflow.add_edge("search_news", "generate_theme")
workflow.add_edge("generate_theme", "search_sub_themes")

for i in range(5):
    workflow.add_edge("search_sub_themes", f"write_section_{i}")
    workflow.add_edge(f"write_section_{i}", "aggregate")

workflow.add_edge("aggregate", "editor")
workflow.add_edge("editor", END)

graph = workflow.compile()

In [ ]:
from util import display_graph

display_graph(graph)

In [12]:
# keyword = input("Enter a keyword for the newsletter: ")
keyword = "대한민국 윤석열 대통령 탄핵심판"
inputs = {"keyword": keyword}
async for output in graph.astream(inputs, stream_mode="updates"):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("----")
        print(value)
    print("\n---\m")

Output from node 'search_news':
----
{'article_titles': ["South Korea's Yoon defends his martial law decree as impeachment ruling nears - The Sun Chronicle", 'Prosecutors compare South Korean president Yoon to a dictator as impeachment trial nears end - NBC News', 'South Korea’s Impeached President Yoon Faces Final Court Hearing - Bloomberg', 'South Korea’s Yoon Suk Yeol defends martial law decree at impeachment trial - AOL', "South Korea's Yoon defends his martial law decree as impeachment ruling nears - ABC News", 'South Korean President Yoon Suk Yeol brought to court for final hearing of impeachment trial - The Audubon County Advocate Journal', 'S. Korea opposition urges court remove Yoon over martial law - The Daily Record', 'South Koreans to Rally as Yoon’s Impeachment Verdict Nears - Bloomberg', 'South Korea’s Yoon defends his martial law decree as impeachment trial nears end - YourCentralValley.com', 'South Korean President Yoon Suk Yeol brought to court for final hearing of imp

In [ ]:
# 대한민국 윤석열 대통령 탄핵심판